In [1]:
import os
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision.transforms as Tr
from torch.optim.lr_scheduler import StepLR

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\ProgramData\Anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
class ConvLayer(nn.Module):
    
    def __init__(self, cin, parameters, activation=nn.LeakyReLU(0.1)):
        
        super().__init__()
        
        cout        = parameters['cout']
        kernel_size = parameters['kernel_size']
        padding     = parameters['padding']
        stride      = parameters['stride'] 
        
        conv = nn.Conv2d(in_channels=cin, 
                         out_channels=cout, 
                         kernel_size=kernel_size, 
                         padding=padding,
                         stride=stride,
                         bias=False)
        bn =  nn.BatchNorm2d(cout)
        self.layer = nn.Sequential(*[conv, bn, activation])
    
    def forward(self, x):
        
        return self.layer(x)
    
class FCLayers(nn.Module):
    
    def __init__(self, cin, fc_architecture, activation=nn.LeakyReLU(0.1)):
        
        super().__init__()
        
        S = fc_architecture['S']
        B = fc_architecture['B']
        C = fc_architecture['C']
        c_hidden = fc_architecture['c_hidden']
        
        layer_list = []
        layer_list.append(nn.Flatten())
        layer_list.append(nn.Linear(cin*S*S, c_hidden))
        layer_list.append(activation)
        layer_list.append(nn.Linear(c_hidden, (C+5*B)*S*S))
        self.layers = nn.Sequential(*layer_list)
        
    def forward(self, x):
        
        return self.layers(x)

class YOLO1(nn.Module):
    
    def __init__(self, cnn_architecture, fc_architecture, cin=3, activation=nn.LeakyReLU(0.1)):
        
        super().__init__()
        
        layer_list = []
        c_input = cin
        
        for element in cnn_architecture:
            if element['type']=='cnn_block':
                for n in range(element['repeat']):
                    for parameters in element['layers']:
                        layer_list.append(ConvLayer(c_input, parameters, activation))
                        c_input = parameters['cout']
            if element['type']=='maxpool':
                pool = nn.MaxPool2d(kernel_size=element['parameters']['kernel_size'], stride=element['parameters']['stride'])
                layer_list.append(pool)
                    
        self.layers = nn.Sequential(*layer_list)
        
        self.fc_layers = FCLayers(c_input, fc_architecture, activation=activation)
    
    def forward(self, x):
        
        out = self.layers(x)
        out = self.fc_layers(out)

        return out

In [3]:
path_data = './data/PascalVOC_YOLO/'

In [4]:
class VOCDataset(Dataset):
    
    def __init__(self, path, file, S=7, C=20, imagesize=448):
        
        self.data = pd.read_csv(os.path.join(path, file), header=None).values
        self.sample_path = os.path.join(os.path.join(path, 'images'))
        self.label_path = os.path.join(os.path.join(path, 'labels'))
        self.S = S
        self.C = C
        self.imagesize = imagesize
    
    def __len__(self):
        
        return len(self.data)
    
    def __getitem__(self,index):
        
        img = Image.open(os.path.join(self.sample_path, self.data[index,0]))
        img = img.resize((self.imagesize,self.imagesize))
        sample = np.asanyarray(img)
        sample = sample/255
        sample = torch.tensor(np.transpose(sample, (2,0,1)), dtype=torch.float)
        label_raw = np.loadtxt(os.path.join(self.label_path, self.data[index,1]))
        
        label = torch.zeros((self.S, self.S, self.C + 5))

        for n in range(len(label_raw)):
            c = int(label_raw[n,0])
            loc = (self.S*label_raw[n,1:3]).astype(int)
            box_center = self.S*label_raw[n,1:3] - loc
            box_size = self.S*label_raw[n,3:5]
            box = torch.tensor(np.concatenate((box_center, box_size)), dtype=torch.float)
            label[loc[1], loc[0], c] = 1
            label[loc[1], loc[0], self.C] = 1
            label[loc[1], loc[0], self.C+1:] = box

        
        return sample, label

In [9]:
batch_size = 100

train_dataset = VOCDataset(path_data, 'train.csv', imagesize=448)

test_dataset = VOCDataset(path_data, 'test.csv', imagesize=448)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

-[source #1](hhttps://arxiv.org/abs/1506.02640)

-[source #2](https://www.youtube.com/watch?v=n9_XyCGr-MI&list=PLy5rjn5-uSPAKe2PfszYRqNY7JJC45P1d&index=7&t=1920s)

In [7]:
cnn_architecture = []

element_1 = {'type': 'cnn_block',
             'layers':[{'cout':64,
                        'kernel_size':7,
                        'padding': 3,
                        'stride':2}],
            'repeat': 1}

cnn_architecture.append(element_1)

element_2 = {'type': 'maxpool',
             'parameters':{'kernel_size':2,
                           'stride': 2}} 

cnn_architecture.append(element_2)

element_3 = {'type': 'cnn_block',
             'layers':[{'cout':192,
                        'kernel_size':3,
                        'padding': 1,
                        'stride':1}],
            'repeat': 1}

cnn_architecture.append(element_3)

element_4 = {'type': 'maxpool',
             'parameters':{'kernel_size':2,
                           'stride': 2}} 

cnn_architecture.append(element_4)

element_5 = {'type': 'cnn_block',
             'layers':[{'cout':128,
                        'kernel_size':1,
                        'padding': 0,
                        'stride':1},
                      {'cout':256,
                        'kernel_size':3,
                        'padding': 1,
                        'stride':1},
                      {'cout':256,
                        'kernel_size':1,
                        'padding': 0,
                        'stride':1},
                      {'cout':512,
                        'kernel_size':3,
                        'padding': 1,
                        'stride':1}],
            'repeat': 1}

cnn_architecture.append(element_5)

element_6 = {'type': 'maxpool',
             'parameters':{'kernel_size':2,
                           'stride': 2}} 

cnn_architecture.append(element_6)

element_7 = {'type': 'cnn_block',
             'layers':[{'cout':256,
                        'kernel_size':1,
                        'padding': 0,
                        'stride':1},
                      {'cout':512,
                        'kernel_size':3,
                        'padding': 1,
                        'stride':1}],
            'repeat': 4}

cnn_architecture.append(element_7)

element_8 = {'type': 'cnn_block',
             'layers':[{'cout':512,
                        'kernel_size':1,
                        'padding': 0,
                        'stride':1},
                      {'cout':1024,
                        'kernel_size':3,
                        'padding': 1,
                        'stride':1}],
            'repeat': 1}

cnn_architecture.append(element_8)

element_9 = {'type': 'maxpool',
             'parameters':{'kernel_size':2,
                           'stride': 2}} 

cnn_architecture.append(element_9)

element_10 = {'type': 'cnn_block',
             'layers':[{'cout':512,
                        'kernel_size':1,
                        'padding': 0,
                        'stride':1},
                      {'cout':1024,
                        'kernel_size':3,
                        'padding': 1,
                        'stride':1}],
            'repeat': 2}

cnn_architecture.append(element_10)

element_11 = {'type': 'cnn_block',
             'layers':[{'cout':1024,
                        'kernel_size':3,
                        'padding': 1,
                        'stride':1},
                      {'cout':1024,
                        'kernel_size':3,
                        'padding': 1,
                        'stride':2},
                      {'cout':1024,
                        'kernel_size':3,
                        'padding': 1,
                        'stride':1},
                      {'cout':1024,
                        'kernel_size':3,
                        'padding': 1,
                        'stride':1}],
            'repeat': 1}

cnn_architecture.append(element_11)

fc_architecture ={'S':7, 'B':2, 'C':20, 'c_hidden': 496}

In [8]:
model = YOLO1(cnn_architecture, fc_architecture)